# Solição para cotar e comparar preço de produtos na web

### Importando módulo e lista de produtos

In [40]:
import pandas as pd
from selenium import webdriver
import re

produto = pd.read_excel('Produtos.xlsx').fillna('-')
display(produto)


,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,-,-
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,-,-


In [109]:
#Entrar no site da Amazon
driver = webdriver.Chrome()
driver.get('https://www.amazon.com.br/LG-50UN731C-Assistente-Virtual-Bluetooth/dp/B08FJCW8H3/ref=dp_prsubs_1?pd_rd_i=B08FJCW8H3&th=1')
#Descobrir o objeto onde está o preç
preco_amazon = driver.find_element_by_css_selector('span.apexPriceToPay').text
preco_amazon = float(re.sub(r'[A-Z$\.]','', preco_amazon).replace(',','.'))
print(preco_amazon)
#Armazenar o preço em uma variável


2636.1


C:\Users\dougl\AppData\Local\Temp/ipykernel_11416/4204344012.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  preco_amazon = driver.find_element_by_css_selector('span.apexPriceToPay').text


2636.10


<class 'float'>
